# Импорт необходимых библиотек

In [4]:
import os
from dotenv import load_dotenv
from minio import Minio
import boto3
from pyspark.sql import SparkSession
import requests
import logging

# Подключение к Minio через библиотеку boto3 

In [31]:
load_dotenv()

# Получаем переменные
endpoint = os.getenv("MINIO_ENDPOINT") # для airflow нужно будет переписать чтобы значения брались из Variables
access_key = os.getenv("MINIO_ACCESS_KEY")
secret_key = os.getenv("MINIO_SECRET_KEY")

# Подключаемся к MinIO
s3 = boto3.client(
    's3',
    endpoint_url=f"http://{endpoint}",
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key

)

# Проверяем список бакетов
try:
    buckets = s3.list_buckets()
    print("Подключение к MinIO через boto3 успешно. Список бакетов:")
    for b in buckets['Buckets']:
        print(f"  - {b['Name']}")
except Exception as e:
    print("Ошибка подключения к MinIO:", e)

Подключение к MinIO через boto3 успешно. Список бакетов:
  - project


# Подключение к Minio через библиотеку Minio 

In [41]:
client = Minio(
    endpoint=os.getenv("MINIO_ENDPOINT"), # для airflow нужно будет переписать чтобы значения брались из Variables
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False  
)

# Проверяем список бакетов
try:
    buckets = client.list_buckets()
    print("Подключение к MinIO через MinIO успешно. Список бакетов:")
    for b in buckets:
        print(f' - {b.name}')
except Exception as e:
    print("Ошибка подключения к MinIO:", e)

Подключение к MinIO через MinIO успешно. Список бакетов:
 - project


# Проверка SparkSession 

In [43]:
spark = SparkSession.builder \
    .appName("MinIO Test") \
    .getOrCreate()

df = spark.range(10)
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



# Запуск SparkSession с различными параметрами 

**Имя приложения, которое будет отражено в Spark UI и логах**

appName("SparkProject")

**Локальный режим (может быть yarn, mesos, kubernetes)**

master("local[*]") 

**Количество партиций, используемых при операциях shuffle (например, groupBy, join). По умолчанию = 200.
Для небольших наборов данных можно уменьшить, чтобы не было лишних партиций.**

config("spark.sql.shuffle.partitions", "200") 

**Объем памяти, выделяемый каждому executor'у. Чем больше данных — тем больше памяти потребуется.**

config("spark.executor.memory", "4g")

**Объем памяти для драйвера — основной управляющий процесс, запускающий задачи и отслеживающий их выполнение.**

config("spark.driver.memory", "2g") 

**Количество ядер CPU, которые каждый executor может использовать. Больше ядер — больше параллелизма, но стоит учитывать общую нагрузку.**

config("spark.executor.cores", "4")

**Включает динамическое выделение executors в зависимости от загрузки. Позволяет Spark автоматически масштабироваться: добавлять и убирать executors в зависимости от нужд приложения. Требует настройки shuffle service.**

config("spark.dynamicAllocation.enabled", "true")

**Указывает директорию для checkpoint'ов (резервного сохранения состояния, особенно в streaming-приложениях).**

config("spark.checkpoint.dir", "/path/to/checkpoint/dir")

**Путь к директории, где Spark SQL будет хранить управляемые таблицы (при использовании saveAsTable, например).**

config("spark.sql.catalogImplementation", "hive")


**Указывает, что Spark должен использовать Hive Catalog (а не встроенный In-Memory метастор). Требует наличие Hive или его имитации (например, Derby DB).**

.enableHiveSupport()


In [5]:
# spark = SparkSession.builder \
#     .appName("SparkProject") \ 
#     .master("local[*]") \ 
#     .config("spark.sql.shuffle.partitions", "200") \ 
#     .config("spark.executor.memory", "4g") \ 
#     .config("spark.driver.memory", "2g") \ 
#     .config("spark.executor.cores", "4") \ 
#     .config("spark.dynamicAllocation.enabled", "true") \ 
#     .config("spark.checkpoint.dir", "/path/to/checkpoint/dir") \ 
#     .config("spark.sql.warehouse.dir", "/path/to/warehouse/dir") \ 
#     .config("spark.sql.catalogImplementation", "hive") \ 
#     .getOrCreate()

# Получаем данные по API

In [6]:
def get_api_data(env_name):
    load_dotenv()
    # Получаем ссылку из файла .env
    api_url = os.getenv(env_name) # для airflow нужно будет переписать чтобы значения брались из Variables
    # Проверка наличия ссылки
    if not api_url:
        logging.error(f'Указанный URL не задан в файле .env') # для airflow нужно будет переписать чтобы значения брались из Variables
        return None
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            json_str = json.dumps(data, ensure_ascii=False, indent=2)
            logging.info('Данные успешно загружены')
            return json_str
        else:
            logging.error(f'Error: {response.status_code} - Данные не загружены')
            return None
    except:
        logging.error(f'Error{e} - Данные не загружены')
        raise



# Сохраняем JSON в MinIO

In [ ]:
def save_json_to_minio(env_name):
    data = get_api_data(env_name)
    
# Написать функцию, чтобы записать в Minio JSON
# Проверить в какой бакет пишем перезаписью
# Через Spark читать файл
    

In [ ]:
spark = SparkSession.builder \
    .appName("SparkProject") \ 
    .master("local[*]") \ 
    .config("spark.sql.shuffle.partitions", "200") \ 
    .config("spark.driver.memory", "2g") \ 
    .config("spark.dynamicAllocation.enabled", "true") \ 
    .getOrCreate()